#### This notebook is now primarily used as a scrap notebook for quick code-testing purposes.

In [12]:
import datetime
datetime.datetime.today().date().strftime('%Y-%m-%d')

'2020-06-05'

In [13]:
from google.cloud.logging import Client
import logging

In [ ]:
Client().setup_logging()

In [19]:
import google.cloud.logging
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'D:/Projects/Personal/Current/Rent AVM/rent-avm/scraper/key.json'

# Instantiates a client
client = google.cloud.logging.Client()

# Retrieves a Cloud Logging handler based on the environment
# you're running in and integrates the handler with the
# Python logging module. By default this captures all logs
# at INFO level and higher
client.get_default_handler()
client.setup_logging()

In [21]:
# Imports Python standard library logging
import logging

# The data to log
text = 'Hello, world!'

# Emits the data using the standard logging module
logging.error(text)

Hello, world!


In [94]:
import pandas as pd

In [95]:
df = pd.DataFrame({'a': [1, 2, 3], 'b': [3, 2, 1]})

In [96]:
df.to_csv('test.csv', index=False)

In [123]:
s = pd.DataFrame({'Apsauga_ČarvuotosDurys': 4, 'c': 5}, index=[0])

In [134]:
object_data = {'a': 1, 'b': 2}

In [138]:
d = {'Č':1, 'Ė':2, 'Ž':3}

unidecode.unidecode('čiulpk bybį')

'ciulpk bybi'

In [135]:
keys_old = object_data.keys()
for key in keys_old:
    key_new = key + "_nx"
    object_data[key_new] = object_data[key]
    print(key)
    print(key_new)

del object_data[keys_old]

a
a_nx


RuntimeError: dictionary changed size during iteration

In [133]:
for key in {'a':1, 'b':2}.keys():
    print(key)

a
b


In [132]:
import unidecode
unidecode.unidecode('Apsauga_ĖarvuotosDurys')

'Apsauga_EarvuotosDurys'

In [130]:
unaccented_string

'Apsauga_SarvuotosDurys'

In [124]:
s.to_gbq('rent_avm.test', project_id='rent-avm', if_exists='replace', progress_bar=False)

In [112]:
app.add_url_rule()

In [113]:
len('Apsauga_ŠarvuotosDurys')

22

In [ ]:
gcloud functions deploy scraper --runtime python37 --trigger-http --region europe-west1 --memory 2048 

In [111]:
from flask import Flask, request

if __name__ == '__main__':

    app = Flask(__name__)

    app.add_url_rule('/scrape', 'scrape', scrape, methods=['POST', 'GET'], defaults={'request': request})

    app.run(host='localhost', port=8080, debug=False)

In [ ]:
import requests
import proxyscrape

collector = proxyscrape.create_collector('collector', 'http')


In [74]:
proxy_obj = collector.get_proxy({'code': ('lt', 'lv', 'pl', 'de'), 'anonymous': True})
proxy = proxy_obj.host + ":" + proxy_obj.port
print(proxy)
session = requests.session()
session.proxies = {
    'http': proxy,
    'https': proxy
}


# Check if the connection works.
session.get('http://httpbin.org/ip').json()['origin']

77.252.26.71:57348


ProxyError: HTTPConnectionPool(host='77.252.26.71', port=57348): Max retries exceeded with url: http://httpbin.org/ip (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000026D25259BB0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond')))

'46.20.59.243:47497'

'195.123.208.164'

In [43]:
proxy.host

'190.103.178.13'

In [ ]:
proxy

In [28]:
!pip install proxyscrape

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
def get_proxies():
    proxy_url = 'https://premproxy.com/list/ip-port/1.htm'
    out = requests.get(proxy_url)
    return out

In [3]:
out = get_proxies()

In [10]:
import pandas as pd

In [27]:
BeautifulSoup(out.content)#.find_all('li')

<!DOCTYPE html>
<html><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, user-scalable=no" name="viewport"/>
<link href="/a-img/favicon.png" rel="shortcut icon" type="image/x-icon"/>
<link href="/a-img/favicon.png" rel="icon" type="image/x-icon"/>
<link href="/a-css/topstyle.css" rel="stylesheet"/><meta content="Sorted by Anonymity Type Proxy List in IP:Port format. Page #1" name="description"/>
<title>Sorted by Anonymity Type Proxy List (IP:Port) | Page #1</title>
<link href="/css/http-lists/8be09b03.css" rel="stylesheet" type="text/css"/>
<script src="/js1/jq.js"></script><script src="/js/d3858.js"></script></head>
<body>
<div id="menu">
<nav class="navbar" id="primaryMenu">
<div class="container">
<div class="navbar-header">
<button class="navbar-toggle collapsed" data-target="#primaryNavbar" data-toggle="collapse" type="button"> <span class="sr-only">Toggle navigation</span> <span class="icon-bar"></span> <span class="icon-bar"></span> <span class="i

In [93]:
class test:
    def __init__(self):
        
        self.config = {"proxies": [
    "192.121.232.96:80", "37.123.164.8:8888", "178.128.206.2:80", "84.15.143.111:32829", "82.135.148.201:8081"
  ]}
        self.max_retries = 3
        
        logging.basicConfig(
            level=logging.INFO,
            format="%(asctime)s [%(levelname)s] %(message)s",
            handlers=[
                logging.StreamHandler()
            ]
        )
        pass
    
    def get_proxy_session(self):
        """
        Creates a requests.session object using proxy ports. Automatically switches to a different proxy in case of an Exception.

        TODO: Automatically update proxy list.
        
        Returns
        -------
        requests.session object
        """
    
        correct_output = False
        retries = 0
        while (not correct_output) and (retries < self.max_retries):
            try:
                # Build a proxy session from the config file. Selects random item from list.
                proxy_number = random.randint(0, len(self.config['proxies']) - 1)
                session = requests.session()
                session.proxies = {
                    'http': self.config['proxies'][proxy_number],
                    'https': self.config['proxies'][proxy_number]
                }
                
                print(self.config['proxies'][proxy_number])
                # Set additional Tor session parameters.
                session.headers = UserAgent().random
                
                # Check if the connection works. 
                output = session.get('http://httpbin.org/ip')
                
                # If no ProxyError occurs, report number of proxies and the connected ip.
                logging.info('Currently there are {} potentially working proxies for use.'.format(len(self.config['proxies'])))
                logging.info('Built a session at IP {}.'.format(output.json()['origin']))
                correct_output = True
                
            except Exception as e:
                # Remove the failed proxy from the list of proxies.
                logging.warning('Proxy at {} failed. Trying a different one.'.format(self.config['proxies'][proxy_number]))
                logging.warning(e)
                del self.config['proxies'][proxy_number]
                retries += 1
        
        return session
    
test().get_proxy_session()

192.121.232.96:80


2020-06-02 12:27:50,394 [INFO] Currently there are 5 potentially working proxies for use.
2020-06-02 12:27:50,395 [WARNING] Proxy at 192.121.232.96:80 failed. Trying a different one.
2020-06-02 12:27:50,396 [WARNING] Expecting value: line 1 column 1 (char 0)


84.15.143.111:32829


2020-06-02 12:27:53,971 [INFO] Currently there are 4 potentially working proxies for use.
2020-06-02 12:27:53,972 [INFO] Built a session at IP 84.15.143.111.


In [ ]:
correct_output = False
        retries = 0
        while (not correct_output) or (retries < self.max_retries):
            try:
                # Scrape the main page to get the total number of pages.
                page = self.session.get(url)
                soup = BeautifulSoup(page.content, 'html.parser')

                # Extract all page number buttons to find the maximum digit value.
                page_number_buttons = str(soup.find_all(class_=self.config['html_tags']['page_number_button']))
                page_number_buttons = re.findall('\d+', page_number_buttons)

                total_pages = max([int(number) for number in page_number_buttons])

                correct_output = True
                return total_pages

            except Exception as e:
                logging.warning('Exception occurred at get_number_of_pages:')
                logging.warning(e)

                retries += 1
                self.session = self.restart_tor()

In [26]:
ip_addresses = ['192.121.232.96:80', '37.123.164.8:8888', '178.128.206.2:80', '84.15.143.111:32829', '82.135.148.201:8081']

In [27]:
proxy_index = random.randint(0, len(ip_addresses) - 1)
proxy = {"http": ip_addresses[proxy_index], "https": ip_addresses[proxy_index]}
print(ip_addresses[proxy_index])

session = requests.session()
print(session.get('http://httpbin.org/ip').json()['origin'])


session = requests.session()
print(session.get('http://httpbin.org/ip', proxies=proxy).json()['origin'])

178.128.206.2:80
78.58.222.15
178.128.206.2
